# A1 - OpenJij core interface入門 (core python interface)

このチュートリアルではOpenJijのcore interface (core python interface)の使い方を説明し、簡単な計算のデモンストレーションを行います。

core interfaceは前回までのチュートリアルよりも下部のレイヤーのAPIとなるため、対象読者としては前回までのOpenJijチュートリアルを一通り終えて、イジングモデルやモンテカルロ法などの用語を知っている方を想定しており、

* 最適化問題だけでなくサンプリングや研究用途などより一般的な用途にOpenJijを用いたい
* アニーリングスケジュールの設定や使用するアルゴリズム等を直接触りたい

といった目的に使用できます。

## OpenJij core interface について

前回までのチュートリアルではOpenJijを用いた様々な問題の解き方やベンチマークの取り方などを紹介してきましたが、OpenJijは最下層の部分は統計物理学の数値計算手法であるマルコフ連鎖モンテカルロ法をベースにC++を用いて実装されており、今まで触れてきたpythonモジュールはこのC++インターフェースを直接ラップしたpythonライブラリである**cxxjij**を呼び出す形で実装されています。図にすると次のような包含関係があります。

<img src="hierarchy.png" alt="OpenJij hierarchy"/>

core interfaceを用いることでOpenJij上の全ての機能を使用することができるため、最適化問題のみならず、統計物理学の数値計算ツールとして研究用途で使用することもできます。
また、C++インターフェースを用いることでより高速な演算を行うことができます。
本チュートリアルではpythonインターフェースのcxxjijと、C++インターフェースの両方を紹介します。
インストールにはpipを使用します。

In [7]:
#!pip install openjij
#!pip show openjij

## 問題を投げてみる

チュートリアルとしてまずは変数のサイズが$N=5$の古典スピン ($\sigma = \pm 1$)イジング問題をアニーリングで解いてみましょう。ハミルトニアンは以下のようになります。
\begin{align}
H &= \sum_{i<j}J_{ij}\sigma_i \sigma_j + \sum_{i=1}^{N}h_i \sigma_i \\
\sigma_i &= \pm 1 (i=1 \cdots N)
\end{align}

縦磁場と相互作用が
$h_i = -1 \ \mathrm{for\ } \forall i,\ J_{ij} = -1 \mathrm{for\ } \forall i,j$
だと各スピンは1の値をとった方がエネルギーが低くなるため、$\{\sigma_i\} = \{1,1,1,1,1\}$が最適解となります。この問題を解いてみましょう。
pythonコードを用いた一通りの流れは次のようになります。

> core interfaceはイジング問題に特化したソルバのため、QUBOとの変換は実装されていません。QUBOとの変換を行うには今までのチュートリアルを参照し、core interfaceを呼ぶ前にQUBOからイジング問題へ変換してください。

In [14]:
# core interfaceではopenjijの代わりにcxxjijをインポートします。
import cxxjij as cj

# まず相互作用行列を作成してあげます。Graphモジュールを使います。
import cxxjij.graph as G

# 問題サイズN=5の密結合グラフ(Dense)を定義します。
N = 5
J = G.Dense(N)
# 相互作用を設定してあげます。
for i in range(N):
    for j in range(N):
        #J[i,i]以外に-1を入力
        J[i,j] = 0 if i == j else -1.0

# 縦磁場を設定してあげます。
for i in range(N):
    J[i] = -1

# 続いてGraphから計算を行うためのSystemを作成します。
import cxxjij.system as S

# 今回は通常の古典モンテカルロ計算のシステムを使用します。
# (Eigenは)
system = S.make_classical_ising_Eigen(J.gen_spin(), J)
# アニーリングスケジュールを設定します。Utilityモジュールを使用します。
import cxxjij.utility as U
schedule = U.make_classical_schedule_list(0.1, 100, 10, 10)

# 実際にアニーリングを走らせます。Algorithmモジュールを使用します。
# モンテカルロステップの更新方法として単純なSingleSpinFlipを用います。
import cxxjij.algorithm as A
A.Algorithm_SingleSpinFlip_run(system, schedule)

# 結果を取得します。Resultモジュールにあるget_solutionを用います。
import cxxjij.result as R
print("The solution is {}.".format(R.get_solution(system)))

The solution is [1, 1, 1, 1, 1].


出てきた答えが$[1,1,1,1,1]$になったのが確認できます。
低レイヤーのAPIのため、設定する項目は多いですがその分詳細な設定が可能となります。

## モジュール一覧

コード例に出てきたように、OpenJij core interfaceでは主に`graph`, `system`, `algorithm`などのモジュールから成り立っており、それぞれのモジュールを組み合わせることで様々な種類、アルゴリズムを用いてイジングモデルを計算することが可能になっています。また新たにアルゴリズムを実装する際に拡張が容易であるという特徴を備えています。次章以降で詳細な説明を行います。

### Graph

イジングハミルトニアンの係数$J_{ij}$を保持するためのモジュールです。基本的に密結合 (全てのJijが0以外の値を持つモデルに適している)を扱う`Dense`と疎結合 (Jijの多くの値が0であるモデルに適している)`Sparse`の2種類が存在します。また、`Sparse`から継承された`Square` (正方格子)、`Chimera` (D-Waveのキメラグラフ)も備えています。内部のC++実装ではそれぞれの構造に合わせて最適なデータ構造で実装されています。

### System

`system`ではモンテカルロ等の計算における、現在のシステムの状態を保持するためのデータ構造が定義されています。具体的には

- 古典イジングモデル (スピン配列)
- 横磁場イジングモデル (トロッター分解も含んだスピン配列)
- GPU実装古典、量子イジングモデル

等が定義されています。モンテカルロ法を始めとする計算手法には極めて様々な手法がある (もしくは今後新しい手法が開発されていく)ため、OpenJijでは各々の計算手法に対応するデータ構造とアルゴリズム、また計算結果の取得インターフェースを分離することにより、様々なアルゴリズムを追加することが容易に行えるように設計されています。

### Updater

どのような手法で`system`を更新していくかを定義します。具体的には
- SingleSpinFlip Update
- SwendsenWang Update

などの手法が実装されています。

> core python interfaceでは`algorithm`と統合されています。

### Algorithm

`updater`を用いてどのようなスケジュールでアニーリングアルゴリズムを実行するか等、アルゴリズムを実行する役割を担います。

### Result

`system`からスピン配位などの情報を得るために使用されます。